<a href="https://colab.research.google.com/github/Georgeman31/std_projects/blob/main/NLP/QuestionAnswering_ProtasovEA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Импорт библиотек

In [ ]:
!pip install datasets transformers evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 90.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 14.9 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
import pandas as pd
import numpy as np
import evaluate

import torch
import transformers
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForQuestionAnswering

import datasets
from datasets import load_dataset

from sklearn.model_selection import train_test_split
import textwrap
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


# Загрузка данных

In [ ]:
dataset = load_dataset("sberquad")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/45328 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5036 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/23936 [00:00<?, ? examples/s]

Dataset sberquad downloaded and prepared to /root/.cache/huggingface/datasets/sberquad/sberquad/1.0.0/62115d937acf2634cfacbfee10c13a7ee39df3ce345bb45af7088676f9811e77. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

# Подготовка модели

In [ ]:
model_checkpoint = "DeepPavlov/rubert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    """
    Tokenizes the questions and context, converts answer offset and length
    into start and end token positions

    Parameters
    ----------
    examples (dict): data instance

    Returns
    ----------
    inputs (dict): tokenized questions, context, attention masks, answers token
    start and end positions
    """
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(questions,
                       examples["context"],
                       max_length=384,
                       truncation="only_second",
                       return_offsets_mapping=True,
                       padding="max_length")

    offset_mapping = inputs["offset_mapping"]
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

  0%|          | 0/46 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/24 [00:00<?, ?ba/s]

In [ ]:
val_idx, test_idx = train_test_split(list(range(len(tokenized_dataset['validation']))),
                                     test_size=0.5,
                                     random_state=0)

dataset_val = torch.utils.data.Subset(tokenized_dataset['validation'], val_idx)
dataset_test = torch.utils.data.Subset(tokenized_dataset['validation'], test_idx)

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from 

# Подготовка функции для расчета метрик качества 

In [ ]:
def get_compute_metrics(dataset, metric):
    """
    Returns a function to compute metrics with defined metric and dataset

    Parameters
    ----------
    dataset (Dataset): dataset with original questions and context to use for
    evaluation
    metric (Callable): function to calculate metric that accepts the two lists
    of question-answers dictionaries as inputs

    Returns
    ----------
    compute_metrics (Callable): function to compute metrics
    """
    def compute_metrics(eval_preds):
        """
        Computes exact match score and the average f1-score

        Parameters
        ----------
        eval_preds (EvalPrediction): model predictions and labels 

        Returns
        ----------
        metrics (dict): exact match score and the average f1-score
        """
        y_pred = np.argmax(eval_preds[0], -1).T

        f1_score = 0
        exact_match = 0
        for data, pred in zip(dataset, y_pred):
            # Convert answer start and end into characters positions in
            # original text using the offset mapping list
            start_char = data['offset_mapping'][pred[0]][0]
            end_char = data['offset_mapping'][pred[1]][1]
            # Create predictions and references dictionaries for metric function
            predictions = [{'prediction_text': data['context'][start_char:end_char],
                            'id': str(data['id'])}]
            references = [{'answers': data['answers'],
                           'id': str(data['id'])}]
            results = metric.compute(predictions=predictions,
                                     references=references)
            # Add metric to running sum variable to calculate average after,
            # change outputs from 0-100 range to 0-1 range
            f1_score += results['f1'] / 100
            exact_match += results['exact_match'] / 100
        # Calculate the average
        f1_score /= len(dataset)
        exact_match /= len(dataset)
        return {'f1': f1_score, 'exact_match': exact_match}

    return compute_metrics


# Get a function to compute metrics to use for evaluation
compute_metrics = get_compute_metrics(dataset=dataset_val,
                                      metric=evaluate.load("squad"))

# Обучение модели

In [ ]:
training_args = TrainingArguments(output_dir='./',
                                  report_to='none',
                                  evaluation_strategy="steps",
                                  eval_steps=300,
                                  logging_steps=300,
                                  save_steps=300,
                                  optim="adamw_torch",
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=16,
                                  per_device_eval_batch_size=16,
                                  warmup_steps=200,
                                  lr_scheduler_type='cosine',
                                  num_train_epochs=1,
                                  load_best_model_at_end=True,
                                  metric_for_best_model='eval_f1')

trainer = Trainer(model,
                  training_args,
                  train_dataset=tokenized_dataset['train'],
                  eval_dataset=dataset_val,
                  compute_metrics=compute_metrics)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: offset_mapping, id, context, title, question, answers. If offset_mapping, id, context, title, question, answers are not expected by `BertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 45328
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2833
  Number of trainable parameters = 177264386


Step,Training Loss,Validation Loss,F1,Exact Match
300,2.188400,1.961628,0.764183,0.567117
600,2.002500,1.805864,0.762036,0.558777
900,1.862300,1.758861,0.799167,0.600477
1200,1.800300,1.688237,0.803284,0.604051
1500,1.753300,1.655990,0.803716,0.604845
1800,1.689000,1.640018,0.800751,0.601668
2100,1.665400,1.623948,0.815362,0.624305
2400,1.704900,1.617589,0.810275,0.615965
2700,1.648300,1.616858,0.813386,0.619539


***** Running Evaluation *****
  Num examples = 2518
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: offset_mapping, id, context, title, question, answers. If offset_mapping, id, context, title, question, answers are not expected by `BertForQuestionAnswering.forward`,  you can safely ignore this message.


Step,Training Loss,Validation Loss


Saving model checkpoint to ./checkpoint-300
Configuration saved in ./checkpoint-300/config.json
Model weights saved in ./checkpoint-300/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2518
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: offset_mapping, id, context, title, question, answers. If offset_mapping, id, context, title, question, answers are not expected by `BertForQuestionAnswering.forward`,  you can safely ignore this message.
Saving model checkpoint to ./checkpoint-600
Configuration saved in ./checkpoint-600/config.json
Model weights saved in ./checkpoint-600/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2518
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: offset_mapping, id, context, title, question, answers. If offse

TrainOutput(global_step=2833, training_loss=1.8051572100164357, metrics={'train_runtime': 4428.604, 'train_samples_per_second': 10.235, 'train_steps_per_second': 0.64, 'total_flos': 8883044541997056.0, 'train_loss': 1.8051572100164357, 'epoch': 1.0})

# Оценка качества

In [ ]:
# Расчет метрик на тестовом датасете
compute_test_metrics = get_compute_metrics(dataset=dataset_test,
                                           metric=evaluate.load("squad"))

# Расчет предиктов на тестовом датасете и их сохранение
model.eval()
with torch.inference_mode():
    start_logits, end_logits = [], []
    for item in tqdm(dataset_test):
        input_ids = torch.tensor(item['input_ids']).unsqueeze(0).to('cuda')
        attention_mask = torch.tensor(item['attention_mask']).unsqueeze(0).to('cuda')
        token_type_ids = torch.tensor(item['token_type_ids']).unsqueeze(0).to('cuda')

        output = model(input_ids=input_ids,
                       token_type_ids=token_type_ids,
                       attention_mask=attention_mask)

        start_logits.append(output['start_logits'].squeeze(0).cpu().numpy())
        end_logits.append(output['end_logits'].squeeze(0).cpu().numpy())

    # Преобразование предиктов в необходимый формат 
    predictions = ((np.array(start_logits), np.array(end_logits)), -1)

# Расчет и визуализация метрик
metrics = compute_test_metrics(predictions)
print(f'\nF1-score on test dataset: {metrics["f1"]}')

100%|██████████| 2518/2518 [01:14<00:00, 33.61it/s]



F1-score on test dataset: 0.8062495417682518


In [ ]:
model.eval()
with torch.inference_mode():

    for i in [0, 100, 200]:
        example = tokenized_dataset['test'][i]
        input_ids = torch.tensor(example['input_ids']).unsqueeze(0).to('cuda')
        token_type_ids = torch.tensor(example['token_type_ids']).unsqueeze(0).to('cuda')
        attention_mask = torch.tensor(example['attention_mask']).unsqueeze(0).to('cuda')

        output = model(input_ids=input_ids,
                       token_type_ids=token_type_ids,
                       attention_mask=attention_mask)

        pred = torch.stack([output['start_logits'],
                            output['end_logits']], dim=1).squeeze(0)
        pred = torch.argmax(pred, -1).T

        start_char = example['offset_mapping'][pred[0]][0]
        end_char = example['offset_mapping'][pred[1]][1]

        print(f'\t\t\t\t\tExample №{i+1}.')
        print(textwrap.fill(f'Context: \t{example["context"]}',
                            initial_indent='',
                            subsequent_indent='\t\t'))
        print(textwrap.fill(f'Question: \t{example["question"]}',
                            initial_indent='',
                            subsequent_indent='\t\t'))
        print(textwrap.fill(f'Answer: \t{example["context"][start_char:end_char]}',
                            initial_indent='',
                            subsequent_indent='\t\t'))
        print(f'\n')

					Example №1.
Context:        Многоклеточный организм — внесистематическая категория
		живых организмов, тело которых состоит из многих клеток, большая
		часть которых (кроме стволовых, например, клеток камбия у растений)
		дифференцированы, то есть различаются по строению и выполняемым
		функциям. Следует отличать многоклеточность и колониальность. У
		колониальных организмов отсутствуют настоящие дифференцированные
		клетки, а следовательно, и разделение тела на ткани. Граница между
		многоклеточностью и колониальностью нечёткая. Например, вольвокс
		часто относят к колониальным организмам, хотя в его колониях есть
		чёткое деление клеток на генеративные и соматические. Кроме
		дифференциации клеток, для многоклеточных характерен и более высокий
		уровень интеграции, чем для колониальных форм. Многоклеточные
		животные, возможно, появились на Земле 2,1 миллиарда лет назад,
		вскоре после кислородной революции .
Question:       У каких организмов отсутствуют настоящие
		дифференцир